In [1]:
%matplotlib qt
import time
import sys
import math
import numpy as np
import pandas as pd
import MySQLdb
from datetime import datetime
import calendar
import holidays
import matplotlib.pyplot as plt
sys.path.insert(0, '/Users/kenleejr92/energy_market_project/scripts/MySQL_scripts')
from Query_ERCOT_DB import Query_ERCOT_DB
import cPickle as pickle
from datetime import datetime
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.mllib.linalg import Matrix, Matrices
import pyspark.mllib.linalg.distributed as pydist
from pyspark.sql import SQLContext
from pyspark.mllib.stat import Statistics
from sklearn.cluster import AffinityPropagation
import mpld3
import re
from sets import Set
mpld3.enable_notebook()

class LMP_Query(Query_ERCOT_DB):

    def __init__(self):
        self.SPPs = ['HB_BUSAVG', 
                     'HB_HOUSTON', 
                     'HB_HUBAVG', 
                     'HB_NORTH', 
                     'HB_SOUTH', 
                     'HB_WEST', 
                     'LZ_AEN', 
                     'LZ_CPS', 
                     'LZ_HOUSTON', 
                     'LZ_LCRA', 
                     'LZ_NORTH', 
                     'LZ_RAYBN', 
                     'LZ_SOUTH',  
                     'LZ_WEST']   
        self.node_dict = {}
        self.table_list = []
        self.df = None
        self.CRR_nodes = None
        self.table_boundaries = {'table0':('0001', 'BLUEMD1_8X'),
                                 'table1':('BLUEMD1_8Z', 'CHT_M'),
                                 'table2':('CHT_X', 'DUKE_8405'),
                                 'table3':('DUKE_8505', 'ELEVEE_E8'),
                                 'table4':('ELEVEE_W8', 'GREENLK_L_A'),
                                 'table5':('GREENLK_L_B', 'KEETER'),
                                 'table6':('KEITH', 'L_CEDAHI8_1Y'),
                                 'table7':('L_CEDAHI8_1Z', 'MOSES_1G'),
                                 'table8':('MOSES_2G', 'PHR_8135'),
                                 'table9':('PHR_8140', 'SANDOW1_8Y'),
                                 'table10':('SANDOW_4G', 'TCN7225_BUS'),
                                 'table11': ('TCN7230_BUS', 'VENSW_1777'),
                                 'table12':('VENSW_1785', '_WC_V_C')
                                 }
        for i in range(0,13):
            Query_ERCOT_DB.c.execute("""SHOW columns FROM DAM_LMP%s""" % i)
            result = [r[0] for r in Query_ERCOT_DB.c.fetchall()[2:]]
            self.table_list.append(result)
            for node in result:
                self.node_dict[node] = i
        
    
    def get_CRR_nodes(self):
        Query_ERCOT_DB.c.execute("""SELECT DISTINCT Sink FROM crr_ownership ORDER BY Sink""")
        nodes = list(Query_ERCOT_DB.c.fetchall())
        CRR_prefixes = [r[0] for r in nodes]
        pattern = re.compile('.*_')
        matching_patterns = Set()
        for idx, node in enumerate(CRR_prefixes):
            matches = re.findall(pattern, node)
            if matches: matching_patterns.add(matches[0][:-1])
        # flatten list of lists      
        all_nodes = [item for sublist in self.table_list for item in sublist] + self.SPPs
        self.CRR_nodes = []
        for pattern in matching_patterns:
            pattern2 = re.compile('(.*%s.*)' % pattern)
            for node in all_nodes:
                if re.search(pattern2, node):
                    self.CRR_nodes.append(node)
        return self.CRR_nodes
    
    def query_single_node(self, node):
        s="""SELECT delivery_date, hour_ending, %s from DAM_LMP%s order by delivery_date, hour_ending""" % (node, self.node_dict[node])
        Query_ERCOT_DB.c.execute(s)
        result = list(Query_ERCOT_DB.c.fetchall())
        fresult = []
        for r in result:
            temp = ()
            date = r[0]
            time = str(int(r[1].split(":")[0])-1)
            dt = datetime.strptime(date + " " + time, "%Y-%m-%d %H")
            for x in r[2:]:
                if x == None: x = 0
                temp = temp + (float(x),)
            r = (dt,) + temp
            fresult.append(r)
        self.df = pd.DataFrame(data=[f[1:] for f in fresult], index=[r[0] for r in fresult], columns=[node])
        return self.df
        
    def query(self, nodes, start_date='2011-01-01', end_date='2015-12-31'):
        node_string=''
        for node in nodes:
            node_string = node_string + node + ',' + ' '
        node_string = node_string[:-2]
        s = """SELECT DAM_LMP0.delivery_date, DAM_LMP0.hour_ending, %s 
                                    from DAM_LMP0 join DAM_LMP1 on (DAM_LMP0.delivery_date = DAM_LMP1.delivery_date and DAM_LMP0.hour_ending = DAM_LMP1.hour_ending) 
                                    join DAM_LMP2 on (DAM_LMP0.delivery_date = DAM_LMP2.delivery_date and DAM_LMP0.hour_ending = DAM_LMP2.hour_ending)
                                    join DAM_LMP3 on (DAM_LMP0.delivery_date = DAM_LMP3.delivery_date and DAM_LMP0.hour_ending = DAM_LMP3.hour_ending)
                                    join DAM_LMP4 on (DAM_LMP0.delivery_date = DAM_LMP4.delivery_date and DAM_LMP0.hour_ending = DAM_LMP4.hour_ending)
                                    join DAM_LMP5 on (DAM_LMP0.delivery_date = DAM_LMP5.delivery_date and DAM_LMP0.hour_ending = DAM_LMP5.hour_ending)
                                    join DAM_LMP6 on (DAM_LMP0.delivery_date = DAM_LMP6.delivery_date and DAM_LMP0.hour_ending = DAM_LMP6.hour_ending)
                                    join DAM_LMP7 on (DAM_LMP0.delivery_date = DAM_LMP7.delivery_date and DAM_LMP0.hour_ending = DAM_LMP7.hour_ending)
                                    join DAM_LMP8 on (DAM_LMP0.delivery_date = DAM_LMP8.delivery_date and DAM_LMP0.hour_ending = DAM_LMP8.hour_ending)
                                    join DAM_LMP9 on (DAM_LMP0.delivery_date = DAM_LMP9.delivery_date and DAM_LMP0.hour_ending = DAM_LMP9.hour_ending)
                                    join DAM_LMP10 on (DAM_LMP0.delivery_date = DAM_LMP10.delivery_date and DAM_LMP0.hour_ending = DAM_LMP10.hour_ending)
                                    join DAM_LMP11 on (DAM_LMP0.delivery_date = DAM_LMP11.delivery_date and DAM_LMP0.hour_ending = DAM_LMP11.hour_ending)
                                    join DAM_LMP12 on (DAM_LMP0.delivery_date = DAM_LMP12.delivery_date and DAM_LMP0.hour_ending = DAM_LMP12.hour_ending)
                                    join DAM_SPPs on (DAM_LMP0.delivery_date = DAM_SPPs.delivery_date and DAM_LMP0.hour_ending = DAM_SPPs.hour_ending)
                                    where DAM_LMP0.delivery_date > "%s" and DAM_LMP0.delivery_date < "%s" order by DAM_LMP0.delivery_date, DAM_LMP0.hour_ending;""" % (node_string, start_date, end_date)
        Query_ERCOT_DB.c.execute(s)
        result = list(Query_ERCOT_DB.c.fetchall())
        fresult = []
        for r in result:
            temp = ()
            date = r[0]
            time = str(int(r[1].split(":")[0])-1)
            dt = datetime.strptime(date + " " + time, "%Y-%m-%d %H")
            for x in r[2:]:
                if x == None: x = 0
                temp = temp + (float(x),)
            r = (dt,) + temp
            fresult.append(r)
        self.df = pd.DataFrame(data=[f[1:] for f in fresult], index=[r[0] for r in fresult], columns=nodes)
        return self.df
        
    def get_price(self, node, date, hour_ending):
        for i in range(0,13):
            node = append_n(node)
            if node in self.table_columns['table%s' % i]:
                Query_ERCOT_DB.c.execute("""SELECT %s FROM DAM_LMP%s WHERE delivery_date = "%s" AND hour_ending = \"%s\"""" % (node, i, date, hour_ending))
                result = list(Query_ERCOT_DB.c.fetchall())[0][0]
                return result
        

def append_n(name):
    if name[0] in ['0','1','2','3','4','5','6','7','8','9'] or name == 'LOAD':
        name = 'n' + name
    return name

def dist(x,y):
    cost = 0
    for i in range(len(x)):
        if x[i] == 0 or y[i] == 0:
            continue
        else:
            cost = cost + np.abs(y[i] - x[i])[0]
    return cost

def string_to_date(string_date):
    return datetime.strptime(string_date, "%Y-%m-%d %H")

def date_to_string(date):
    return date.strftime("%Y-%m-%d %H")

def weekday_of_date(date):
    return calendar.day_name[date.weekday()]

def work_day_or_holiday(date):
    us_holidays = holidays.UnitedStates()
    if date in us_holidays or weekday_of_date(date) == "Sunday" or weekday_of_date(date) == "Saturday":
        return int(1)
    else: return int(0)

/Users/kenleejr92/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:26: DeprecationWarning: the sets module is deprecated


In [2]:
conf = SparkConf().setMaster("local").setAppName("spark_DAM_correlation")
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

In [2]:
qcrr = LMP_Query()

In [3]:
crr_nodes = qcrr.get_CRR_nodes()
f = open('crr_nodes.pkl', 'w+')
pickle.dump(crr_nodes, f)
f.close()

In [57]:
np.random.seed(1111)
f2 = open('crr_nodes.pkl', 'r')
crr_nodes = pickle.load(f2)
f2.close()

In [11]:
x1 = qcrr.query(crr_nodes[0:500],'2011-01-01','2015-12-31').as_matrix()
x2 = qcrr.query(crr_nodes[500:1000], '2011-01-01','2015-12-31').as_matrix()
x3 = qcrr.query(crr_nodes[1000:1500], '2011-01-01','2015-12-31').as_matrix()
x4 = qcrr.query(crr_nodes[1500:2000], '2011-01-01','2015-12-31').as_matrix()
x5 = qcrr.query(crr_nodes[2000:2500], '2011-01-01','2015-12-31').as_matrix()
x6 = qcrr.query(crr_nodes[2500:3000], '2011-01-01','2015-12-31').as_matrix()
x7 = qcrr.query(crr_nodes[3000:], '2011-01-01','2015-12-31').as_matrix()
x8 = np.concatenate((x1,x2), axis=1)
x8 = np.concatenate((x8,x3), axis=1)
x8 = np.concatenate((x8,x4), axis=1)
x8 = np.concatenate((x8,x5), axis=1)
x8 = np.concatenate((x8,x6), axis=1)
x8 = np.concatenate((x8,x7), axis=1)

In [16]:
f1 = open('crr_node_prices.csv', 'w+')
np.savetxt('crr_node_prices.csv', x8, delimiter=',', fmt='%10.5f')
f1.close()

3588

In [4]:
x_df = pd.DataFrame.from_csv('/Users/kenleejr92/energy_market_project/crr_node_correlation_matrix.csv', header=None, index_col=None)
x = x_df.as_matrix()
for i, item in enumerate(x[:,0]):
    x[i,0] = float(item[1:])
for i, item in enumerate(x[:,3587]):
    x[i,3587] = float(item[:-1])
    

In [5]:
correlation_matrix = np.abs(x)
af = AffinityPropagation(affinity='precomputed').fit(correlation_matrix)

In [6]:
col_dict = {}
for i, col in enumerate(crr_nodes):
    col_dict[i] = col

In [7]:
cluster_list = [[] for i in range(af.cluster_centers_indices_.size)]
for idx, item in enumerate(af.labels_):
    cluster_list[item].append(col_dict[idx])
exemplars = []
for idx, cluster in enumerate(cluster_list):
    if len(cluster)>=5:
        exemplars.append(col_dict[idx])
f = open('cluster_list.pkl', 'w+')
pickle.dump(cluster_list, f)
f.close()
f = open('exemplars.pkl', 'w+')
pickle.dump(exemplars, f)
f.close()

In [54]:
f = open('exemplars.pkl', 'r')
exemplars = pickle.load(f)
f.close()
df = qcrr.query(exemplars)
ax = df.plot()
ax.set_xlabel('Hour')
ax.set_ylabel('Dollar/MWh')
ax.legend_.remove()
plt.show()
f = open('exemplar_dataframe.pkl', 'w+')
pickle.dump(df, f)
f.close()

In [11]:
e = exemplars[0]
dfe = df[e]
feature_MIs = []
idx_wout_1st_week = None
print('feature selection for %s' % e)
past_index = [24,25,47,48]
for c in df.columns:
    features = []
    targets = []
    feature_labels = []
    for dt, s in dfe.iteritems():
        temp_features = []
        pred_hour_index = dfe.index.get_loc(dt)
        if type(pred_hour_index) == slice: continue
        if pred_hour_index - 48 >= 0:
            categorical_features = np.array([work_day_or_holiday(dt), dt.hour, dt.weekday(), dt.month])
            targets.append(dfe[pred_hour_index])
            for i in past_index:
                feature_labels += [c + '_' + str(i)]
                temp_features += [df[c].iloc[pred_hour_index-i]]
            features.append(temp_features)
    features = np.array(features)
    targets = np.array(targets)
    time, lag = features.shape
    for i in range(lag):
        f_series = features[:,i]
        f_max = np.max(f_series)
        f_min = np.min(f_series)
        features[:,i] = (f_series-f_min)/(f_max-f_min)
    t_max = np.max(targets)
    t_min = np.min(targets)
    targets = (targets-f_min)/(f_max-f_min)
    t_med = np.median(targets)
    for k,p in enumerate(targets):
        if p >= t_med:
            targets[k] = 1.0
        else: targets[k] = 0.0
    for i in range(lag):
        f_med = np.median(features[:,i])
        for k,p in enumerate(features[:,i]):
            if p >= f_med:
                features[:,i][k] = 1.0
            else: features[:,i][k] = 0.0
    Pt_1 = np.sum(targets)/targets.shape[0]
    Pt_0 = 1 - Pt_1
    for i in range(lag):
        Pf_1 = np.sum(features[:,i])/features[:,i].shape[0]
        Pf_0 = 1 - Pf_1
        f0_t0 = 0.0
        f1_t0 = 0.0
        f0_t1 = 0.0
        f1_t1 = 0.0
        for k,p in enumerate(features[:,i]):
            zero=np.float64(0.0)
            one=np.float64(1.0)
            if (targets[k]==zero) and (p==zero):
                f0_t0 += 1.0
            if (targets[k]==zero) and (p==one):
                f1_t0 += 1.0
            if (targets[k]==one) and (p==zero):
                f0_t1 += 1.0
            if (targets[k]==one) and (p==one):
                f1_t1 += 1.0
        P00 = f0_t0/features[:,i].shape[0]
        P10 = f1_t0/features[:,i].shape[0]
        P01 = f0_t1/features[:,i].shape[0]
        P11 = f1_t1/features[:,i].shape[0]
        if P00==0 or P10==0 or P01==0 or P11 ==0:
            MI_i = 0;
        else: MI_i = P00*math.log(P00/(Pf_0*Pt_0),2)+\
               P10*math.log(P10/(Pf_1*Pt_0),2)+\
                P01*math.log(P01/(Pf_0*Pt_1),2)+ P11*math.log(P11/(Pf_1*Pt_1),2)
        feature_MIs.append((feature_labels[i], MI_i))
    print(c)


feature selection for NCARBIDE_E_2
NCARBIDE_E_2
NCARBIDE_E_3
NCARBIDE_E_4
BL_MILIT_001
BL_MILIT_BLT
BL_MILIT_L_A
BRYN_MILLICZ
EBWINDMILL28
EB_MILTON
HAMILMB1X
HAMILMB1Y
HAMILMB1Z
HAMILMB2
HAMILTON
HAMILTON_L_B
HAMILTON_V_A
HAMILTON_V_B
HAMILTO_0001
MILDRED2_9
MILDRED2_9X
MILE_L_A
MILE_V_A
MILITARY_V_A
MILITARY_V_B
MILLCR_L_A
MILO_L_A
RWMILLER
RWMILLER1
RWMILLER2
TNHAMILTON0Y
QALSW_0001
QALSW_0100
QALSW_AG3
QALSW_GT2
EXNMEANS_8Z
EXN_69A
EXN_EPS_1
EXN_EPS_2
EXN_G10
EXN_G12
EXN_G13
EXN_G14
LNCRK2_871_2
HLSES_8620
HLSES_8621
HLSES_8637
HLSES_KL
HLSES_KS
HLSES_KY
OGSES_K
OGSES_KE
OGSES_KH
SWEC_0001
GUADG_GAS1
GUADG_GAS3
GUADG_GAS4
GUADG_STM5
GUADG_STM6
GUADG_V_A
WHCCS_E_1
WHCCS_E_2
R_STEAM_2
STEAM1A_0004
STEAM_A901
STEAM_KTK
STEAM_LD04
DIB_DIB_G1
DIB_DIB_G2
RSKMN_KU
FORSAN_8
L_SANDHI8_1Y
L_SANMAR8_1X
L_SANMAR8_1Z
L_SANSAS9_1Z
MONSAN__138R
ODESSANO_8S
ODESSANO_8T
PLEASANTON9X
PLEASANT_L_B
PLEASANT_MOB
PLEASANT_V_A
PLEASANT_V_B
R_SANDHSYD_1
SANANDRS_9
SANA_L_A
SANA_V_A
SANDHILL_8
SANDHSYD
SAN

In [12]:
f = open('feature_MIs.pkl','w+')
pickle.dump(feature_MIs, f)
f.close()

In [39]:
feature_MIs.sort(key=lambda tup: tup[1], reverse=True) 
selected = filter(lambda x: x[1]>0.45, feature_MIs)

'NCARBIDE_E_2'

In [ ]:
df = qcrr.query(exemplars)

In [53]:
targets = []
features = []
for pred_hour_index, dt in enumerate(df.index):
    if pred_hour_index - 24 >= 0:
        categorical_features = [work_day_or_holiday(dt), dt.hour, dt.weekday(), dt.month]
        node_names = []
        targets.append(df[exemplars[0]].iloc[pred_hour_index])
        for s in selected:
            node_names.append(s[0][:-3])
        numerical_features = df[node_names].iloc[pred_hour_index-24].as_matrix()
        f = np.concatenate([numerical_features, categorical_features])
        features.append(f)

In [55]:
f=open('features.pkl', 'w+')
pickle.dump(features, f)
f.close()
f=open('targets.pkl', 'w+')
pickle.dump(targets, f)
f.close()